# Calculadora de Facturación Quina (Versión Jupyter)
Este cuaderno implementa la lógica de facturación extraída en el módulo `QuinaLogic.py`, permitiendo realizar los cálculos de manera interactiva y transparente.

### Configuración del Sistema:
- **Búsqueda Automática:** El sistema localizará automáticamente los archivos necesarios en las rutas estándar.
- **Validación de Rutas:** Se realizarán comprobaciones para asegurar la integridad de los datos de entrada.

In [ ]:
import pandas as pd
import os
import sys

# === INICIALIZACIÓN Y BÚSQUEDA DE RECURSOS ===
def localizar_sistema_quina():
    """Localiza el módulo QuinaLogic en las rutas probables del sistema."""
    rutas_candidatas = [
        os.getcwd(),                                            # Directorio de trabajo actual
        os.path.join(os.getcwd(), "Quina"),                 # Subdirectorio 'Quina'
        r"C:\Users\A365\Documents\MisNotebooks\Quina",     # Ruta estándar de despliegue
        "/Quina/",                                          # Ruta raíz (Entornos Linux/JupyterHub)
        r"C:\Quina"                                         # Ruta raíz Windows
    ]
    
    for ruta in rutas_candidatas:
        if os.path.exists(os.path.join(ruta, "QuinaLogic.py")):
            return ruta
    return None

ruta_base = localizar_sistema_quina()

if ruta_base:
    print(f"✅ Sistema localizado exitosamente en: {ruta_base}")
    if ruta_base not in sys.path:
        sys.path.append(ruta_base)
else:
    print("❌ ERROR: No se pudo localizar el módulo 'QuinaLogic'. Verifique la instalación.")

# Importación del Módulo de Lógica
try:
    from QuinaLogic import QuinaCalculator
    print("✅ Módulo QuinaCalculator importado y listo para usar.")
except ImportError as e:
    print(f"❌ ERROR CRÍTICO: Fallo al importar la librería lógica: {e}")

In [ ]:
# === CONFIGURACIÓN DE ARCHIVOS DE ENTRADA ===
# Detección automática del directorio de archivos
directorio_archivos = None
if ruta_base:
    posibles_directorios = [
        os.path.join(ruta_base, "Archivos"),
        os.path.join(ruta_base, "..", "Archivos"),
        ruta_base
    ]
    for d in posibles_directorios:
        if os.path.exists(os.path.join(d, "RDC.xlsx")):
            directorio_archivos = d
            break

if directorio_archivos:
    print(f"✅ Directorio de datos detectado: {directorio_archivos}")
    RUTA_RDC = os.path.join(directorio_archivos, "RDC.xlsx")
    RUTAS_DDC = [
        os.path.join(directorio_archivos, "DDC1.xlsx"),
        os.path.join(directorio_archivos, "DDC2.xlsx")
    ]
else:
    print("⚠️ ADVERTENCIA: No se detectaron los archivos automáticamente.")
    print("Se utilizarán nombres de archivo por defecto (esperando que estén en la misma carpeta).")
    # Configuración manual por defecto
    RUTA_RDC = "RDC.xlsx" 
    RUTAS_DDC = ["DDC1.xlsx", "DDC2.xlsx"]

print(f"Archivo RDC configurado: {RUTA_RDC}")

In [ ]:
# === EJECUCIÓN DEL PROCESAMIENTO ===
try:
    if 'QuinaCalculator' in locals() and os.path.exists(RUTA_RDC):
        calculadora = QuinaCalculator()
        print("⏳ Iniciando procesamiento de datos...")
        
        # Ejecutar lógica de negocio
        resumen_resultados = calculadora.process_data(RUTA_RDC, RUTAS_DDC)
        
        print("\n✅ Cálculo finalizado correctamente.")
        print("Resumen Ejecutivo:")
        for metrica, valor in resumen_resultados.items():
            print(f"  - {metrica}: {valor:,.0f}")
    else:
        print("❌ ERROR: No es posible iniciar el cálculo. Verifique la existencia de los archivos de entrada.")
except Exception as e:
    print(f"❌ Excepción durante el procesamiento: {e}")

In [ ]:
# === VISUALIZACIÓN DE AUDITORÍA ===
if 'calculadora' in locals() and calculadora.df_detalle is not None:
    print("Visualizando las primeras 5 filas del detalle de auditoría generado:")
    display(calculadora.df_detalle.head())
else:
    print("No hay datos de auditoría disponibles para mostrar.")

In [ ]:
# === GENERACIÓN DE REPORTES ===
# Determinar ruta de salida
if 'RUTA_RDC' in locals() and os.path.exists(os.path.dirname(RUTA_RDC)):
    DIRECTORIO_SALIDA = os.path.dirname(RUTA_RDC)
else:
    DIRECTORIO_SALIDA = os.getcwd()

ARCHIVO_SALIDA = os.path.join(DIRECTORIO_SALIDA, "FACTURA_FINAL_JUPYTER.xlsx")

try:
    if 'calculadora' in locals() and calculadora.df_detalle is not None:
        contenido_excel = calculadora.generate_excel_report()
        with open(ARCHIVO_SALIDA, "wb") as f:
            f.write(contenido_excel)
        print(f"\n📄 Reporte generado exitosamente:")
        print(f"   {ARCHIVO_SALIDA}")
    else:
        print("⚠️ No se generó el reporte debido a falta de datos procesados.")
except Exception as e:
    print(f"❌ Error al escribir el archivo de reporte: {e}")